In [1]:
import os
import sys
import time

sys.path.append(os.path.abspath("../.."))

### Test 1


In [ ]:
from typing import Dict

from primeGraph.buffer.factory import History, LastValue
from primeGraph.checkpoint.storage.local_storage import LocalStorage
from primeGraph.constants import END, START
from primeGraph.graph.executable import Graph
from primeGraph.models.state import GraphState


# Define our state model
class ProcessState(GraphState):
    status: LastValue[str]
    results: History[Dict[str, float]]


# Initialize state and graph with local storage and chain_id
chain_id = "process_workflow_v1"
state = ProcessState(status="", results={})
storage = LocalStorage()
graph = Graph(state=state, checkpoint_storage=storage, chain_id=chain_id)


# Define processing nodes
@graph.node()
def initialize_process(state):
    time.sleep(0.5)  # Simulate work
    return {"status": "initializing"}


@graph.node()
def process_data_1(state):
    time.sleep(0.5)  # Simulate work
    return {"status": "processing_1", "results": {"accuracy": 0.85, "step": 1.0}}


@graph.node(interrupt="after")
def process_data_2(state):
    time.sleep(0.5)  # Simulate work
    return {"status": "processing_2", "results": {"accuracy": 0.92, "step": 2.0}}


@graph.node()
def finalize(state):
    time.sleep(0.5)  # Simulate work
    return {"status": "completed"}


# Create the workflow
graph.add_edge(START, "initialize_process")
graph.add_edge("initialize_process", "process_data_1")
graph.add_edge("process_data_1", "process_data_2")
graph.add_edge("process_data_2", "finalize")
graph.add_edge("finalize", END)

# Compile and execute
graph.compile()
graph.visualize()

In [ ]:
graph.execute()

In [ ]:
storage.list_checkpoints(chain_id)

graph.checkpoint_storage.list_checkpoints(chain_id)

In [ ]:
storage.list_checkpoints(chain_id)

### Test 2 (LocalStorage)


In [ ]:
from primeGraph.buffer.factory import History
from primeGraph.checkpoint.local_storage import LocalStorage
from primeGraph.constants import END, START
from primeGraph.graph.executable import Graph
from primeGraph.models.state import GraphState


class StateForTestWithHistory(GraphState):
    execution_order: History[str]


state = StateForTestWithHistory(execution_order=[])
storage = LocalStorage()
graph = Graph(state=state, checkpoint_storage=storage)


@graph.node()
def task1(state):
    print("task1")
    time.sleep(0.5)
    return {"execution_order": "task1"}


@graph.node()
def task2(state):
    print("task2")
    time.sleep(0.5)
    return {"execution_order": "task2"}


@graph.node()
def task3(state):
    print("task3")
    time.sleep(1)
    return {"execution_order": "task3"}


@graph.node()
def task4(state):
    print("task4")
    time.sleep(2)
    print("task4 done")

    return {"execution_order": "task4"}


@graph.node()
def task5(state):
    print("task5")
    time.sleep(1)
    return {"execution_order": "task5"}


@graph.node(interrupt="before")
def task6(state):
    print("task6")
    return {"execution_order": "task6"}


graph.add_edge(START, "task1")
graph.add_edge("task1", "task2")
graph.add_edge("task2", "task3")
graph.add_edge("task2", "task4")
graph.add_edge("task2", "task5")
graph.add_edge("task4", "task6")
graph.add_edge("task3", "task6")
graph.add_edge("task5", "task6")
graph.add_edge("task6", END)
graph.compile()

graph.visualize()

In [ ]:
from rich import print as rprint

rprint(graph.detailed_execution_path)

In [ ]:
from rich import print as rprint

graph._convert_execution_plan()
rprint(graph.execution_plan)

In [ ]:
graph.start()

In [ ]:
graph.state.execution_order

In [ ]:
storage.list_checkpoints(graph.chain_id)

In [ ]:
graph.visualize()

In [ ]:
graph.state

In [ ]:
# start a new chain just to test the load from checkpoint
new_chain_id = graph.start()
print(new_chain_id)

In [ ]:
from rich import print as rprint

rprint(storage._storage)

In [ ]:
print("current_chain_id", graph.chain_id)
print("saved_chain_id", chain_id)
graph.load_from_checkpoint(chain_id)
print("after load chain_id", graph.chain_id)

graph.resume()
assert all(
    task in graph.state.execution_order
    for task in ["task1", "task2", "task3", "task4", "task5", "task6"]
)

In [ ]:
graph.load_from_checkpoint(chain_id)
graph.state.execution_order

In [ ]:
graph.state

In [ ]:
graph.state.execution_order

### Test 3 (PostgreSQLStorage)


In [ ]:
from tiny_graph.buffer.factory import History
from tiny_graph.checkpoint.postgresql import PostgreSQLStorage
from tiny_graph.constants import END, START
from tiny_graph.graph.executable import Graph
from tiny_graph.models.state import GraphState


class StateForTestWithHistory(GraphState):
    execution_order: History[str]


state = StateForTestWithHistory(execution_order=[])
storage = PostgreSQLStorage.from_config(
    **{
        "host": "localhost",
        "port": 5432,
        "user": "tiny_graph",
        "password": "tiny_graph",
        "database": "tiny_graph",
    }
)

assert storage.check_schema(), "Schema is not valid"

graph = Graph(state=state, checkpoint_storage=storage)


@graph.node()
def task1(state):
    print("task1")
    time.sleep(0.5)
    return {"execution_order": "task1"}


@graph.node()
def task2(state):
    print("task2")
    time.sleep(0.5)
    return {"execution_order": "task2"}


@graph.node()
def task3(state):
    print("task3")
    time.sleep(1)
    return {"execution_order": "task3"}


@graph.node()
def task4(state):
    print("task4")
    time.sleep(2)
    print("task4 done")

    return {"execution_order": "task4"}


@graph.node()
def task5(state):
    print("task5")
    time.sleep(1)
    return {"execution_order": "task5"}


@graph.node(interrupt="before")
def task6(state):
    print("task6")
    return {"execution_order": "task6"}


graph.add_edge(START, "task1")
graph.add_edge("task1", "task2")
graph.add_edge("task2", "task3")
graph.add_edge("task2", "task4")
graph.add_edge("task2", "task5")
graph.add_edge("task4", "task6")
graph.add_edge("task3", "task6")
graph.add_edge("task5", "task6")
graph.add_edge("task6", END)
graph.compile()

graph.visualize()

In [ ]:
chain_id = graph.start()
print(chain_id)
assert all(
    task in graph.state.execution_order
    for task in ["task1", "task2", "task3", "task4", "task5"]
), "tasks are not in there"
assert len(storage.list_checkpoints(graph.chain_id)) == 4  # n + 1 due to interrupt

In [ ]:
graph.state.execution_order

In [ ]:
storage.list_checkpoints(graph.chain_id)

In [ ]:
graph.state

In [ ]:
# start a new chain just to test the load from checkpoint
new_chain_id = graph.start()
print(new_chain_id)

In [ ]:
from rich import print as rprint

rprint(storage._storage)

In [ ]:
print("current_chain_id", graph.chain_id)
print("saved_chain_id", chain_id)
graph.load_from_checkpoint(chain_id)
print("after load chain_id", graph.chain_id)

graph.resume()
assert all(
    task in graph.state.execution_order
    for task in ["task1", "task2", "task3", "task4", "task5", "task6"]
)

In [ ]:
graph.load_from_checkpoint(chain_id)
graph.state.execution_order

In [ ]:
graph.state

In [ ]:
graph.state.execution_order

### Outro


In [22]:


from primeGraph.buffer.factory import History, LastValue

from primeGraph.checkpoint.local_storage import LocalStorage
from primeGraph.constants import END, START
from primeGraph.graph.executable import Graph
from primeGraph.models.state import GraphState

class StateForTestWithHistory(GraphState):
  execution_order: History[str]
  
storage = LocalStorage()

def generate_graph():
    state = StateForTestWithHistory(execution_order=[])
    graph = Graph(state=state, checkpoint_storage=storage)

    @graph.node()
    def task1(state):
        print("task1")
        return {"execution_order": "task1"}

    @graph.node()
    def task2(state):
        print("task2")
        return {"execution_order": "task2"}

    @graph.node()
    def task3(state):
        print("task3")
        return {"execution_order": "task3"}

    @graph.node()
    def task4(state):
        print("task4")

        return {"execution_order": "task4"}

    @graph.node()
    def task5(state):
        print("task5")
        return {"execution_order": "task5"}

    @graph.node(interrupt="before")
    def task6(state):
        print("task6")
        return {"execution_order": "task6"}

    graph.add_edge(START, "task1")
    graph.add_edge("task1", "task2")
    graph.add_edge("task2", "task3")
    graph.add_edge("task2", "task4")
    graph.add_edge("task2", "task5")
    graph.add_edge("task4", "task6")
    graph.add_edge("task3", "task6")
    graph.add_edge("task5", "task6")
    graph.add_edge("task6", END)
    graph.compile()

    return graph


In [ ]:
graph = generate_graph()
graph.visualize()

In [ ]:
graph.state_schema

In [ ]:
chain_id = await graph.start_async()

In [ ]:
graph.chain_status

In [ ]:
chain_id = graph.start()
assert all(
task in graph.state.execution_order for task in ["task1", "task2", "task3", "task4", "task5"]
), "tasks are not in there"
assert len(storage.list_checkpoints(graph.chain_id)) == 3  # n + 1 due to interrupt

# start a new chain just to test the load from checkpoint
new_chain_id = graph.start()
assert new_chain_id != chain_id

# loading first chain state
graph = generate_graph()
graph.load_from_checkpoint(chain_id)

# resuming execution
graph.resume()
assert all(
task in graph.state.execution_order for task in ["task1", "task2", "task3", "task4", "task5", "task6"]
), "tasks are not in there"

In [4]:
graph.next_execution_node

### Non serializable object


In [2]:
import os
from typing import Any, Dict

import instructor
from openai import OpenAI
from primeGraph import END, START, Graph
from primeGraph.buffer import History, LastValue
from primeGraph.checkpoint.postgresql import PostgreSQLStorage
from pydantic import BaseModel, Field
from rich import print as rprint
from primeGraph.models.state import GraphState
# from dotenv import load_dotenv


# ENVIRONMENT = os.environ.get("ENVIRONMENT", "dev")
# load_dotenv(f".env.{ENVIRONMENT}")

# NOTION_API_KEY = os.getenv("NOTION_API_KEY")


sys_prompt_extract_page = """
==== OVERALL GUIDANCE =====
You are an expert using Notion API. 

You will be given instruction on how to fill a page in Notion. You should follow the instructions carefully.

You will be working with a database and adding entries to a plan database. 

Keep the scope of the instructions in mind and the pages as atomic but complete as possible.


==== GUIDELINES ON HOW TO ACT =====

- Follow the isntructions very carefully
- Make good use of the Notion API to create the page
- Make pages rich in content but also easy to ready and find the information you need
"""


class NotionInsertionState(GraphState):
    created_pages: History[Any] = Field(default_factory=list)
    instructions: LastValue[Dict[str, Any]] = Field(default_factory=dict)


def get_notion_insertion_graph(
    graph_params: Dict[str, Any],
    graph_state: NotionInsertionState,
):
    notion_graph = Graph(state=graph_state)
    client = instructor.from_openai(OpenAI())

    # Checking required state variables
    if not graph_state.instructions and len(graph_state.instructions) == 0:
        if "instructions" not in graph_params:
            raise ValueError("Instructions of type {step_name: instruction} are required to create a new page")
        else:
            graph_state.instructions = graph_params["instructions"]

    repeated_nodes = list(graph_state.instructions.keys())
    number_parallel_nodes = len(repeated_nodes)

    # Notion initialization
    if "database_id" not in graph_params:
        raise ValueError("database_id is required to create a new page")
    # notion_client = NotionClient(database_id=graph_params["database_id"], api_key=NOTION_API_KEY)
    # page_schema = notion_client.page_schema
    # property_options = notion_client.stringified_property_options

    @notion_graph.node()
    def start(self, state):
        return {}

    @notion_graph.node()
    def extract_page_data(self, state):
        """
        This will be generated base on the number of pages that need to be created
        Pages will be created in parallel
        Based on instructions on the state, each node will have the same name of the instruction key
        We then capture the node name and grab the specific instruction for that page
        """

        class ExtractPageDataResponse(BaseModel):
            notion_page: page_schema = Field(description="The page that you should create")  # type: ignore

        try:
            instruction = state.instructions[self.name]
        except KeyError:
            raise ValueError(f"Instruction for {self.name} not found in state.instructions")

        completion = client.chat.completions.create(
            model="gpt-4o",
            response_model=ExtractPageDataResponse,
            messages=[
                {"role": "system", "content": sys_prompt_extract_page},
                {
                    "role": "user",
                    "content": f""" Create a new page in the database based on the following instructions: {instruction}. 
                                    The database has the following properties, strictly follow them: {property_options}""",
                },
            ],
        )
        rprint(completion.notion_page)
        rprint(state)

        return {
            "created_pages": completion.notion_page,
        }

    @notion_graph.node()
    def submit_to_api(self, state):
        for page in state.created_pages:
            page_object = page.model_dump(exclude_none=True)
            page_object["parent"]["database_id"] = notion_client.database_id
            notion_client.create_page(page_object)
        return {}

    notion_graph.add_edge(START, "start")
    notion_graph.add_repeating_edge(
        "start",
        "extract_page_data",
        "submit_to_api",
        number_parallel_nodes,
        parallel=True if number_parallel_nodes > 1 else False,
        repeat_names=repeated_nodes,
    )
    notion_graph.add_edge("submit_to_api", END)

    notion_graph.compile()

    return notion_graph


In [3]:
graph = get_notion_insertion_graph(
    graph_params={"database_id": "1234567890"},
    graph_state=NotionInsertionState(instructions={"test": "test", "test2": "test2"}))

2025-02-24 23:14:56.368 - instructor - DEBUG - Patching `client.chat.completions.create` with mode=<Mode.TOOLS: 'tool_call'>


In [4]:
graph._get_schema(NotionInsertionState)

In [6]:
graph.state_schema
graph.buffers

{'created_pages': <primeGraph.buffer.history.HistoryBuffer at 0x11882a450>,
 'instructions': <primeGraph.buffer.last_value.LastValueBuffer at 0x119a18f50>}